In [62]:
import pandas as pd
import nltk

nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giuli\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [63]:
categories = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'hope']
categories_dfs = {}
for category in categories:
	categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)

C:\Users\giuli\AppData\Local\Temp\ipykernel_2108\445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_2108\445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  categories_dfs[category] = pd.read_csv(f"./lexicon/{category}-NRC-Emotion-Lexicon.txt", header=None, names=['word', category], sep=None)
C:\Users\giuli\AppData\Local\Temp\ipykernel_2108\445186173.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warni

In [64]:
lexicon = pd.DataFrame(columns=['word'])
for category in categories:
    lexicon = pd.merge(lexicon, categories_dfs[category], on='word', how='outer')
lexicon.dropna(inplace=True) # row with empty string
lexicon.sort_values(by='word', inplace=True)
lexicon

,word,anger,disgust,fear,joy,sadness,surprise,hope
11384,aback,0,0,0,0,0,0,0
11830,abacus,0,0,0,0,0,0,0
1324,abandon,0,0,1,0,1,0,0
663,abandoned,1,0,1,0,1,0,0
609,abandonment,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...
5281,zone,0,0,0,0,0,0,0
5458,zoo,0,0,0,0,0,0,0
7907,zoological,0,0,0,0,0,0,0
13967,zoology,0,0,0,0,0,0,0


In [65]:
#TRAIN_DATA_PATH = "./datasets/WASSA23_essay_level_with_labels_train.tsv"
TRAIN_DATA_PATH = "./datasets/WASSA23_essay_level_dev_preproc.tsv"
#TRAIN_DATA_PATH = "datasets/WASSA23_essay_level_test_preproc.tsv"
train_df = pd.read_csv(TRAIN_DATA_PATH, sep='\t')

In [66]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

preproc_lexicon = {}
lexicon['preproc_word'] = ''
for index, row in lexicon.iterrows():
    lemma = lemmatizer.lemmatize(row['word'])
    stemma = stemmer.stem(lemma)
    lexicon.loc[index, 'preproc_word'] = stemma
    if stemma in preproc_lexicon:
        preproc_lexicon[stemma] += lexicon.loc[index, categories] # |=
    else:
        preproc_lexicon[stemma] = lexicon.loc[index, categories]
preproc_lexicon = pd.DataFrame(preproc_lexicon).T
preproc_lexicon[preproc_lexicon > 0] = 1
preproc_lexicon = preproc_lexicon.loc[(preproc_lexicon!=0).any(axis=1)]
preproc_lexicon.to_csv('./lexicon/preproc_lexicon.csv')
lexicon.to_csv('./lexicon/lexicon.csv', index=False)
preproc_lexicon

,anger,disgust,fear,joy,sadness,surprise,hope
abandon,1,0,1,0,1,1,0
abduct,0,0,1,0,1,1,0
aberr,0,1,0,0,0,0,0
abhor,1,1,1,0,0,0,0
abhorr,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...
youth,1,0,1,1,0,1,0
zani,0,0,0,0,0,1,0
zeal,0,0,0,1,0,1,1
zealou,0,0,0,1,0,0,0


In [67]:
import re # python library to recoginze punctuation and digit
import contractions # python library to recoginze contractions

def remove_punctuations(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_digit(text):
    return re.sub('\d+', '', text)

In [68]:
for category in categories:
	train_df[f'{category}_count'] = [0 for _ in range(len(train_df))]

for index, row in train_df.iterrows(): #TODO: controllare ( on no?)
	for word in row['essay']:
		if word in lexicon['word']:
			for category in categories:
				train_df.loc[index, f'{category}_count'] += lexicon.loc[word][category]

	essay = remove_punctuations(row['essay'])
	essay = remove_digit(essay)
	contracs = [contractions.fix(word) for word in essay.split()]
	lemmas = [lemmatizer.lemmatize(contraction) for contraction in contracs]
	stemmas = [stemmer.stem(lemma) for lemma in lemmas]

	for stemma in stemmas:
		if stemma in preproc_lexicon.index:
			for category in categories:
				train_df.loc[index, f'{category}_count'] += preproc_lexicon.loc[stemma][category]
train_df


,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,...,iri_personal_distress,iri_fantasy,iri_empathatic_concern,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count
0,1,35,How sad is it that this kind of pain and suffe...,68,2,2,1,21,20000,1,...,3.0000,3.143,3.286,3,2,4,3,6,1,0
1,4,35,The article is kind of tragic and hits close t...,79,1,6,3,33,64000,1,...,1.0000,2.429,1.429,4,1,5,3,3,1,0
2,7,213,"I think that these kinds of stories, are sad, ...",68,2,2,1,21,20000,1,...,3.0000,3.143,3.286,1,1,1,6,5,5,4
3,9,213,It's crazy that random accidents like this hap...,84,2,4,1,25,55000,1,...,2.8570,3.571,3.143,2,1,4,5,6,2,1
4,12,78,This story makes me so so sad.... As someone w...,68,2,2,1,21,20000,1,...,3.0000,3.143,3.286,0,0,1,2,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,471,297,"After reading the article, you can't help but ...",70,1,6,1,29,85000,2,...,2.0715,4.143,4.643,4,2,5,3,8,4,1
204,478,53,I am definiltly guilty of wasting food. I buy ...,68,2,2,1,21,20000,2,...,3.0000,3.143,3.286,4,2,2,4,4,1,0
205,484,292,I wish this article had given us more info. We...,68,2,2,1,21,20000,2,...,3.0000,3.143,3.286,5,2,6,2,4,1,2
206,494,218,This story is really horrifying to me. Knowing...,96,2,3,1,27,25000,2,...,2.2860,3.571,3.714,2,3,4,2,3,0,1


In [70]:
for index, row in train_df.iterrows():
    word_count = len(row['essay'].split())
    for category in categories:
	    train_df.loc[index, f'{category}_count'] /= (word_count/100)

train_df
    

,conversation_id,article_id,essay,speaker_id,gender,education,race,age,income,speaker_number,...,iri_personal_distress,iri_fantasy,iri_empathatic_concern,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count
0,1,35,How sad is it that this kind of pain and suffe...,68,2,2,1,21,20000,1,...,3.0000,3.143,3.286,4.761905,3.174603,6.349206,4.761905,9.523810,1.587302,0.000000
1,4,35,The article is kind of tragic and hits close t...,79,1,6,3,33,64000,1,...,1.0000,2.429,1.429,6.349206,1.587302,7.936508,4.761905,4.761905,1.587302,0.000000
2,7,213,"I think that these kinds of stories, are sad, ...",68,2,2,1,21,20000,1,...,3.0000,3.143,3.286,1.754386,1.754386,1.754386,10.526316,8.771930,8.771930,7.017544
3,9,213,It's crazy that random accidents like this hap...,84,2,4,1,25,55000,1,...,2.8570,3.571,3.143,2.777778,1.388889,5.555556,6.944444,8.333333,2.777778,1.388889
4,12,78,This story makes me so so sad.... As someone w...,68,2,2,1,21,20000,1,...,3.0000,3.143,3.286,0.000000,0.000000,1.315789,2.631579,3.947368,1.315789,1.315789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,471,297,"After reading the article, you can't help but ...",70,1,6,1,29,85000,2,...,2.0715,4.143,4.643,5.970149,2.985075,7.462687,4.477612,11.940299,5.970149,1.492537
204,478,53,I am definiltly guilty of wasting food. I buy ...,68,2,2,1,21,20000,2,...,3.0000,3.143,3.286,5.555556,2.777778,2.777778,5.555556,5.555556,1.388889,0.000000
205,484,292,I wish this article had given us more info. We...,68,2,2,1,21,20000,2,...,3.0000,3.143,3.286,8.620690,3.448276,10.344828,3.448276,6.896552,1.724138,3.448276
206,494,218,This story is really horrifying to me. Knowing...,96,2,3,1,27,25000,2,...,2.2860,3.571,3.714,2.325581,3.488372,4.651163,2.325581,3.488372,0.000000,1.162791


In [72]:
train_df.columns

Index(['conversation_id', 'article_id', 'essay', 'speaker_id', 'gender',
       'education', 'race', 'age', 'income', 'speaker_number', 'split',
       'essay_id', 'empathy', 'distress', 'emotion',
       'personality_conscientiousness', 'personality_openess',
       'personality_extraversion', 'personality_agreeableness',
       'personality_stability', 'iri_perspective_taking',
       'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern',
       'anger_count', 'disgust_count', 'fear_count', 'joy_count',
       'sadness_count', 'surprise_count', 'hope_count'],
      dtype='object')

In [71]:
#train_df.to_csv("datasets/train_essay_lev_preproc.tsv", index=False, sep='\t')
train_df.to_csv("datasets/dev_essay_lev_preproc.tsv", index=False, sep='\t')
#train_df.to_csv("datasets/test_essay_lev_preproc.tsv", index=False, sep='\t')